In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

# Sellers

Our goal is to find Sellers that repeatedly underperform vs others, and understand why.  
This will help us shape our recommendations on how to improve Olist's profit margin

<details>
    <summary>🔥 Notebook best practices (reminder) </summary>

From now on, exploratory notebooks are going to get pretty long, and we strongly advise you to follow these notebook principles
- Code your logic so that your Notebook can always be run from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` for intermediary steps when you know you won't need them long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you're between method brackets e.g. `group_by()` to get the docs! Repeat a few times to open it permanently

</details>





## 1 - Code `olist/seller.py` 

- Create the method `get_training_data` in `olist/seller.py` that will return the following DataFrame:

  - `seller_id` (_str_) _the id of the seller **UNIQUE**
  - `seller_city` (_str_) _the city where seller is located_
  - `seller_state` (_str_) _the state where seller is located_
  - `delay_to_carrier` (_float_) _Average delay_to_carrier per seller. Return 0 if the order is delivered before the shipping_limit_date, otherwise the absolute value of the delay
  - `wait_time` (_float_) _Average wait_time (duration of deliveries) per seller_
  - `date_first_sale` (_datetime_) _Date of first sales on Olist_
  - `date_last_sale` (_datetime_) _Date of last sales on Olist_
  - `share_of_five_stars` (_float_) _The share of five star orders for orders in which the seller was involved_
  - `share_of_one_stars` (_float_) _The share of one star orders for orders in which the seller was involved_
  - `review_score` (_float_) _The average review score for orders in which the seller was involved_
  - `n_orders` (_int_) _The number of unique orders the seller was involved with._
  - `quantity` (_int_) _The total number of items sold by this seller_
  - `quantity_per_order`(_float_) _The mean number of items per order for this seller_
  - `sales` (_float_) _The total sales associated with this seller (excluding freight value)_ in BRL
  
Feel free to code all intermediary methods below if you prefer to breakdown the problem step by step.

In [24]:
def get_wait_time(is_delivered = True):
    
        """
        02-01 > Returns a DataFrame with:
        [order_id, wait_time, expected_wait_time, delay_vs_expected, order_status]
        and filtering out non-delivered orders unless specified
        """
        # Hint: Within this instance method, you have access to the instance of
        # the class Order in the variable self, as well as all its attributes

        from olist.data import Olist
        olist = Olist()
        data = olist.get_data()
        orders = data['orders'].copy()

        if is_delivered == True:

            orders = orders[orders['order_status'] == 'delivered']

            #data['orders'].copy()

        dates = [
            'order_purchase_timestamp', 'order_approved_at',
            'order_delivered_carrier_date', 'order_delivered_customer_date',
            'order_estimated_delivery_date'
        ]

        for dat in dates:
            orders[dat] = pd.to_datetime(orders[dat])

        orders['wait_time'] = orders['order_delivered_customer_date'].map(pd.Timestamp.to_julian_date) \
            - orders['order_purchase_timestamp'].map(pd.Timestamp.to_julian_date)

        orders['expected_wait_time'] = orders['order_estimated_delivery_date'].map(pd.Timestamp.to_julian_date) \
            - orders['order_purchase_timestamp'].map(pd.Timestamp.to_julian_date) \

        orders['delay_vs_expected'] = orders['order_delivered_customer_date'].map(pd.Timestamp.to_julian_date) \
            - orders['order_estimated_delivery_date'].map(pd.Timestamp.to_julian_date)

        orders['delay_vs_expected'] = np.where(orders['delay_vs_expected'] < 0,
                                               0, orders['delay_vs_expected'])
        #print(orders.shape)
        return orders[[
            'order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected',
            'order_status'
        ]]



In [101]:
from olist.data import Olist
olist = Olist()
data = olist.get_data()

sellers = data['sellers'][['seller_id','seller_city','seller_state']].copy()
orders = data['orders'].copy()
orders_items = data['order_items'].copy()

#order_review = data['order_items'].copy()
reviews = data['order_reviews'].copy()

#Merges
order_all = orders_items.merge(orders, on ='order_id')
review_all = reviews.merge(orders_items, on ='order_id')

#Convert to date

dats = ['shipping_limit_date','order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date', 'order_delivered_customer_date',
            'order_estimated_delivery_date']

for dat in dats:
    order_all[dat] = pd.to_datetime(order_all[dat])
    
#Create "order_all['delay_to_carrier']"

order_all['delay_to_carrier'] = order_all['order_delivered_carrier_date'].map(pd.Timestamp.to_julian_date) \
            - order_all['order_purchase_timestamp'].map(pd.Timestamp.to_julian_date)


order_all['delay_to_carrier'] = np.where(order_all['order_delivered_carrier_date'] < order_all['shipping_limit_date'], 0, 
                                         order_all['delay_to_carrier'])


#Create "order_all['wait_time']"
order_all['wait_time'] = order_all['order_delivered_customer_date'].map(pd.Timestamp.to_julian_date) \
            - order_all['order_purchase_timestamp'].map(pd.Timestamp.to_julian_date)

#Create "order_all['date_first_sale']"

df_aux = order_all[['seller_id','order_purchase_timestamp']].groupby('seller_id').min().drop_duplicates()
order_all['date_first_sale'] = df_aux['order_purchase_timestamp']

#Create "order_all['date_last_sale']"

df_aux = order_all[['seller_id','order_purchase_timestamp']].groupby('seller_id').max().drop_duplicates()
order_all['date_last_sale'] = df_aux['order_purchase_timestamp']


## Review & orders_items



link = orders_items[['order_id','seller_id']].drop_duplicates()



In [104]:
review_all

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,1,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2018-01-18 15:47:59,185.00,13.63
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,2,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2018-01-18 15:47:59,185.00,13.63
2,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,1,be0dbdc3d67d55727a65d4cd696ca73c,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-06 12:48:39,79.79,8.30
3,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-02-08 10:31:15,149.00,45.12
4,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,1,52c80cedd4e90108bf4fa6a206ef6b03,a1043bafd471dff536d0c462352beb48,2017-04-13 17:55:19,179.99,42.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113317,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42,1,3a33c980b62eb1ef3b8ae61b6fc6fe55,46dc3b2cc0980fb8ec44634e21d2718e,2017-12-07 22:08:00,199.99,9.77
113318,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43,1,8ba0118a487ec8671aed57e5ef846574,17ca9b9e9b9ef8fdb529001b49ebb50f,2018-03-22 10:08:04,215.97,15.59
113319,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13,1,73a7fbf8c1048131f3b531af31bcdf0e,8d956fec2e4337affcb520f56fd8cbfd,2018-06-26 18:15:29,50.95,15.46
113320,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43,1,54caa022e792f1ce31d4a656cecaa802,25cf099de44674fde97473224f9d59ab,2017-12-19 03:59:36,10.00,7.78


In [106]:
#reviews = data['order_reviews'].copy()
#review_all = reviews.merge(orders_items, on ='order_id')

#share_of_five_stars
#df_aux = review_all[['order_id','review_score','seller_id']]

review_all['five_stars'] = np.where(review_all['review_score'] == 5, 1, 0)
review_all['one_stars'] = np.where(review_all['review_score'] == 1, 1, 0)

df_aux =review_all[['seller_id','review_score']].groupby('seller_id').count()



In [113]:
review_all[['seller_id','order_id']].drop_duplicates() #113322 

,seller_id,order_id
0,6d803cb79cc31c41c4c789a75933b3c7,73fc7af87114b39712e6da79b0a377eb
2,8e6d7754bc7e0f22c96d255ebda59eba,a548910a1c6147796b98fdf73dbeba33
3,a1043bafd471dff536d0c462352beb48,f9e4b658b201a9f2ecdecbb34bed034b
4,a1043bafd471dff536d0c462352beb48,658677c97b385a9be170737859d3511b
5,989becdce12ebc39863c2bceab6f3ca1,8e6bfb81e283fa7e4f11123a3fb894f1
...,...,...
113317,46dc3b2cc0980fb8ec44634e21d2718e,22ec9f0669f784db00fa86d035cf8602
113318,17ca9b9e9b9ef8fdb529001b49ebb50f,55d4004744368f5571d1f590031933e4
113319,8d956fec2e4337affcb520f56fd8cbfd,7725825d039fc1f0ceb7635e3f7d9206
113320,25cf099de44674fde97473224f9d59ab,f8bd3f2000c28c5342fedeb5e50f2e75


In [74]:

share_of_five_stars (float) The share of five star orders for orders in which the seller was involved
share_of_one_stars (float) The share of one star orders for orders in which the seller was involved
review_score (float) The average review score for orders in which the seller was involved
n_orders (int) The number of unique orders the seller was involved with.
quantity (int) The total number of items sold by this seller
quantity_per_order(float) The mean number of items per order for this seller
sales (float) The total sales associated with this seller (excluding freight value) in BRL


In [77]:
review_all

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,1,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2018-01-18 15:47:59,185.00,13.63
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,2,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2018-01-18 15:47:59,185.00,13.63
2,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,1,be0dbdc3d67d55727a65d4cd696ca73c,8e6d7754bc7e0f22c96d255ebda59eba,2018-03-06 12:48:39,79.79,8.30
3,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2018-02-08 10:31:15,149.00,45.12
4,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,1,52c80cedd4e90108bf4fa6a206ef6b03,a1043bafd471dff536d0c462352beb48,2017-04-13 17:55:19,179.99,42.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...
113317,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42,1,3a33c980b62eb1ef3b8ae61b6fc6fe55,46dc3b2cc0980fb8ec44634e21d2718e,2017-12-07 22:08:00,199.99,9.77
113318,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43,1,8ba0118a487ec8671aed57e5ef846574,17ca9b9e9b9ef8fdb529001b49ebb50f,2018-03-22 10:08:04,215.97,15.59
113319,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13,1,73a7fbf8c1048131f3b531af31bcdf0e,8d956fec2e4337affcb520f56fd8cbfd,2018-06-26 18:15:29,50.95,15.46
113320,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43,1,54caa022e792f1ce31d4a656cecaa802,25cf099de44674fde97473224f9d59ab,2017-12-19 03:59:36,10.00,7.78


### `get_seller_features`
Returns a DataFrame with: 'seller_id', 'seller_city', 'seller_state'

### `get_seller_delay_wait_time`
Returns a DataFrame with: 'seller_id', 'delay_to_carrier', 'wait_time'

### `get_active_dates`
Returns a DataFrame with 'seller_id', 'date_first_sale', 'date_last_sale'

### `get_review_score`
['seller_id', 'share_of_five_stars', 'share_of_one_stars', 'review_score']

### `get_quantity`
['seller_id', 'n_orders', 'quantity', 'quantity_per_order']

### `get_sales`
['seller_id', 'sales']

### Save your logic into seller.py

### Test your code

In [ ]:
from nbresult import ChallengeResult
from olist.seller import Seller

sellers = Seller().get_training_data()
sellers.head()
data = Seller().get_training_data()
result = ChallengeResult('seller',
shape=data.shape,
columns=sorted(list(data.columns)),
avg_review_score=int(data['review_score'].mean()),
unique_state=sorted(list(data['seller_state'].unique())),
min_wait_time=round(data['wait_time'].min(), 2),
max_wait_time=int(data['wait_time'].max()),
avg_wait_time=int(data['wait_time'].mean()),
avg_delay_carrier=round(data['delay_to_carrier'].mean(), 2),
avg_quantity=int(data['quantity'].mean()),
max_quantity=data['quantity'].max(),
min_quantity=data['quantity'].min(),
avg_sales=int(data['sales'].mean())
)
result.write()
print(result.check())

✅ Once your logic is encoded, commit and push your new file `seller.py`  
✅ Feel free to continue working on this notebook, or make a copy and delete the code above to start section _2 - Exploration_ from a fresh noteook

## 2 - Exploration

### 2.1 - Plots

❓ Let's start with some initial exploratory analysis on sellers distribution:

- Plot the distribution of each numerical variable of the dataset in one large figure
- Do you notice any outliers?
- What's the median of orders per seller? How is the distribution of that variable looking?

In [ ]:
# Your code

----
💡There seems to be a group of sellers which stands out for having very low review scores! Let's investigate graphically:

❓ Using plotly, create a scatterplot of `delay_to_carrier` against `wait_time`, varying bubble size by total `sales` for that seller, and coloring by `review_score`. 

In [ ]:
import plotly.express as px

Feel free to change values `x`, `y`, `color` and `size` to try identify who are the worst sellers

### 2.2 - Model out `review_score` with OLS

❓ Scatter plots have their limits. A more rigorous way to explain sellers' review_score is to **model out the impact of various features on `review_score` with a multivariate-OLS in statsmodels.** 

Create an OLS with only the numerical features of your choice. What are the most impactful ones? Don't forget to standardize your features using our function below if you want to compare the regression coefficients together.

In [ ]:
def standardize(df, features):
    df_standardized = df.copy()
    for f in features:
        mu = df[f].mean()
        sigma = df[f].std()
        df_standardized[f] = df[f].map(lambda x: (x - mu) / sigma)
    return df_standardized

In [ ]:
# Your code

❓ Finally, investigate your model's performance (r-squared) and residuals

In [ ]:
# Your code

### 2.3 - Add seller_state to your analysis

❓ We haven't used information about `seller_state` yet.  
- Create a new OLS model regressing `review_score` on `seller_states` only.
- Analyse your significant features using `return_significative_coef(model)` coded for you in `olist/utils.py`
- What are the best states in terms of `review_score`? 

<details>
    <summary>Hints</summary>
        
Be careful, seller_state is a categorical feature. Use `C(a_cat_feature)` in the formula to indicate categorical variables. It will create one boolean variable `is_cat_feature_xx` **per unique category** 

</details>

In [ ]:
# Your code

☝️ Some states indeed have _signigicantly_ better reviews than others on average. Is it due to some lower `quantity_per_order`, lower `wait_time`, or `delay_to_carrier`? Or is it due to some other factors that we haven't collected data about?

❓ Try to isolate the impact of the `seller_state` from the rest by adding other continuous features to your OLS until seller_states is no longer statistically siginificant.

In [ ]:
# Your code

☝️ After adding `wait_time` to our analysis, none of the 22 dummy variables `is_seller_state_xx` are statistically signigicant:

Given our small dataset (most states have very few sellers):
- We _cannot conclude_ that "some states are inherently better than other for reasons that would be independent of the `wait_time`" 
- In other words, we _cannot reject the hypothesis_ that "seller_state has no impact on review_score, other than through `wait_time`"

🏁 **Congratulations! Commit and push your notebook as well as `seller.py`**